In [33]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "loGu88Z",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "u6p1OlcKS8K0ulvvGBdVKw",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
import os
print "current dir: ", os.getcwd()
os.chdir("/home/foodmap/food101/")
print "current dir: ", os.getcwd()

current dir:  /home/foodmap/food101/notebooks
current dir:  /home/foodmap/food101


In [3]:
from stream2index import streamTweets
from stream2index.Stream2Index import Stream2Index
from elasticsearch import Elasticsearch
import time
from twython import TwythonRateLimitError

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [ ]:
#### PUT CONFIG HERE
CONSUMER_KEY = 'JzAQ0Y2GbBg900GENP444u24K'
CONSUMER_SECRET = '5Mu6TnLR2oC7G9Z87egRzJU2X8vV3mL17qI5jBkL75g3wuLi5c'
ACCESS_TOKEN = '308456550-i6JR7eJo8H0VfV1X0Ujg06YhL9rJOFD3Pgey3zbi'
ACCESS_TOKEN_SECRET = 'J2rY8MCjkDblQpI3LPCXtG7v6nNO7K3YOOZbTASUGiRni'

# Minimal time accepted between two Rate Limit Errors
TOO_SOON = 10
# Time to wait if we receive a Rate Limit Error too soon after a previous one
WAIT_SOME_MORE = 60

In [ ]:
htString = streamTweets.getKeywordsList()

In [ ]:
stream = Stream2Index(CONSUMER_KEY, CONSUMER_SECRET,
                        ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
try:
    # FILTER
    stream.statuses.filter(track=htString)
except TwythonRateLimitError as e:
    # If this error is received after only few calls (10 seconds of calls) wait just a minute
    if time.time() - start < TOO_SOON:
        log('Waiting %s seconds more for resuming download after recurrent rate limit error ...'
            % WAIT_SOME_MORE)
        time.sleep(WAIT_SOME_MORE)
    else:
        log(e, id)
        log('Waiting %s seconds for resuming download after rate limit error ...')
        time.sleep(60)

In [34]:
es = Elasticsearch(['localhost'], http_auth=('elastic', 'changeme'), port=9200)

In [37]:
es.get(index='stream', doc_type='tweet_snippet', id=877436391956860928)

2017-06-21 14:53:28,537 GET http://localhost:9200/stream/tweet_snippet/877436391956860928 [status:200 request:0.004s]


{u'_id': u'877436391956860928',
 u'_index': u'stream',
 u'_source': {u'bounding_box': {u'coordinates': [[[-2.659936, 51.399367],
     [-2.659936, 51.516387],
     [-2.510844, 51.516387],
     [-2.510844, 51.399367]]],
   u'type': u'Polygon'},
  u'coordinates': None,
  u'created_at': u'Wed Jun 21 08:01:52 +0000 2017',
  u'day': 20170621,
  u'id': 877436391956860928,
  u'img_category': u'hamburger',
  u'media_url': u'http://pbs.twimg.com/media/DC1HufGW0AAS1nb.jpg',
  u'text': u'Looking for a commis #chef in #bristol. To start immediately. Please RT #bread #sourdough https://t.co/gSQZCDE7Wb',
  u'username': u'eddiepereztoro'},
 u'_type': u'tweet_snippet',
 u'_version': 1,
 u'found': True}

In [42]:
es.count(index='stream', doc_type='tweet_snippet')

2017-06-23 11:31:53,386 GET http://localhost:9200/stream/tweet_snippet/_count [status:200 request:0.005s]


{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 389}

In [31]:
# es.delete_by_query(index='stream', doc_type='tweet_snippet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

In [25]:
import json
from index.preprocess_tweet import get_image_category
results = es.search(index='stream', doc_type='tweet_snippet', size=100, body={
  'query': {
    'match': {
      'img_category': 'Sushi',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

# print results
# print len(results["hits"]["hits"])


request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

2017-06-21 10:15:13,579 GET http://localhost:9200/stream/tweet_snippet/_search?size=100 [status:200 request:0.005s]


Our sushi rolls 😍 #Takumi #DowntownSac #Izakaya #Sushi #SushiRolls #HappyHour #Dinner #JapaneseFood https://t.co/w1jHqVreef
http://pbs.twimg.com/media/DCzKHN5UQAEC-Qc.jpg
{"guessed":[{"label":"sushi","score":7.0}],"visuallySimilar":[{"id":"sushi/1924555","score":0.38684043,"url":"http://food.isti.cnr.it/foodimages/sushi/1924555.jpg"},{"id":"omelette/1580883","score":0.38625517,"url":"http://food.isti.cnr.it/foodimages/omelette/1580883.jpg"},{"id":"sushi/2152906","score":0.38577515,"url":"http://food.isti.cnr.it/foodimages/sushi/2152906.jpg"},{"id":"sushi/492302","score":0.38289678,"url":"http://food.isti.cnr.it/foodimages/sushi/492302.jpg"},{"id":"sashimi/3291519","score":0.38208473,"url":"http://food.isti.cnr.it/foodimages/sashimi/3291519.jpg"},{"id":"sushi/2120484","score":0.37767214,"url":"http://food.isti.cnr.it/foodimages/sushi/2120484.jpg"},{"id":"caesar_salad/1813477","score":0.37679663,"url":"http://food.isti.cnr.it/foodimages/caesar_salad/1813477.jpg"},{"id":"cheese_plate/2327

### get all docs from index

In [23]:
res = es.search(index="stream", doc_type='tweet_snippet', size=100, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]


2017-06-21 10:11:36,712 GET http://localhost:9200/stream/tweet_snippet/_search?size=100 [status:200 request:0.006s]


[{u'bounding_box': {u'coordinates': [[[172.050587, -42.305574],
     [172.050587, -40.498029],
     [173.304418, -40.498029],
     [173.304418, -42.305574]]],
   u'type': u'Polygon'},
  u'coordinates': None,
  u'created_at': u'Tue Jun 20 21:40:28 +0000 2017',
  u'day': 20170620,
  u'id': 877280009693519872,
  u'img_category': u'fish_and_chips',
  u'media_url': u'http://pbs.twimg.com/media/DCy5gNAU0AAmQqt.jpg',
  u'text': u'\u6628\u65e5\u3075\u3068\u5165\u3063\u305f\u30ec\u30b9\u30c8\u30e9\u30f3\u3001\u3059\u3054\u304f\u7f8e\u5473\u3057\u304b\u3063\u305f\u3051\u3069\u7d50\u69cb\u51f9\u3080\u30ec\u30d9\u30eb\u306e\u4fa1\u683c\u8a2d\u5b9a\u3067\u8349\u3002\u3053\u308c\u3067\u7d046000\u5186\u3002\n#Newzealand #\u8349\n#motueka #dinner https://t.co/7jYSIl8DDu',
  u'username': u'yudaism55'},
 {u'bounding_box': {u'coordinates': [[[18.925169, 47.349448],
     [18.925169, 47.613013],
     [19.334487, 47.613013],
     [19.334487, 47.349448]]],
   u'type': u'Polygon'},
  u'coordinates': None,
  u

In [ ]:
import requests
import json

response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
json_data = json.loads(response.text)
json_data